# 1. Merge all files of sonhdothi.net

In [65]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import sleep
import os
import re

In [66]:
folder_path = "D:\\OneDrive\\KiotViet\\Python_for_work\\KFinance\\CSV_crawl_data"
file_list = os.listdir(folder_path)

merged_df = pd.DataFrame()

# Loop through each file in the folder
for file in file_list:
    if "alonhadat_" in file and file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)

        # Specify the encoding as 'utf-8' or the appropriate encoding for your files
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
            merged_df = pd.concat([merged_df, df], ignore_index=True)
        except UnicodeDecodeError:
            print(f"Error reading file: {file_path}. Skipping.")

print(len(merged_df))

5080


In [67]:
merged_df = merged_df.dropna()
merged_df.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_alonhadat.csv", index=False, mode = 'w', encoding="utf-8-sig")

In [68]:
df = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_alonhadat.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 5080


In [69]:
df = df[~df['Price'].str.contains('Thỏa thuận')]
df = df[~df['Area'].str.contains('KXĐ')]
df = df[~df['Area'].str.contains('90.000.000 m')]
df = df.drop_duplicates()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 4225


**Xử lý diện tích**

In [70]:
df['Area'].head()

0     Diện tích: 24 m\n2
1    Diện tích: 500 m\n2
2    Diện tích: 550 m\n2
3    Diện tích: 100 m\n2
4    Diện tích: 175 m\n2
Name: Area, dtype: object

In [71]:
df['area_value'] = df['Area'].apply(lambda x: float(x.replace('Diện tích: ','').replace('m\n2','').replace(',','.')))

In [72]:
df['area_value']

0        24.0
1       500.0
2       550.0
3       100.0
4       175.0
        ...  
5073     65.0
5074     64.0
5075     80.0
5076     80.0
5077     84.0
Name: area_value, Length: 4225, dtype: float64

**Xử lý giá**

In [73]:
df['Price']

0              Giá: 9 triệu / tháng
1       Giá: 245 ngàn / m2  / tháng
2             Giá: 40 triệu / tháng
3             Giá: 50 triệu / tháng
4             Giá: 85 triệu / tháng
                   ...             
5073         Giá: 110 triệu / tháng
5074         Giá: 100 triệu / tháng
5075          Giá: 59 triệu / tháng
5076         Giá: 110 triệu / tháng
5077          Giá: 55 triệu / tháng
Name: Price, Length: 4225, dtype: object

In [74]:
#Tạo cột 'price_per_m2'

df['price_per_m2'] = df['Price'].apply(lambda x: str(x.replace('Giá: ', '')))
def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) (triệu / tháng)')
price_pattern_ty = re.compile(r'([\d,]+) (tỷ /tháng)')
price_pattern_nghin = re.compile(r'([\d,]+) (ngàn / m2  / tháng)')

#1.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000/Y
df['price_value_trieu'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) is not None else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']


#3.Nếu Price ghi X tỷ/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000000/Y
df['price_value_ty'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern_ty.search(x).group(1)) if price_pattern_ty.search(x) is not None else None)
df['price_per_m2_ty'] = 1000000000 * df['price_value_ty'] / df['area_value']


#3.Nếu Price ghi X triệu và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000/Y
df['price_value_nghin'] = df['price_per_m2'].apply(lambda x: convert_price_string_to_float(price_pattern_nghin.search(x).group(1)) if price_pattern_nghin.search(x) is not None else None)
df['price_per_m2_nghin'] = 1000000 * df['price_value_nghin'] / df['area_value']



In [78]:
#4. Từ các cột "price_per_m2_trieu", "price_per_m2_ty", "price_per_m2_nghin"

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_ty']):
        return row['price_per_m2_ty']
    elif not pd.isna(row['price_per_m2_nghin']):
        return row['price_per_m2_nghin']
    else:
        return np.nan

df['price_per_m2'] = df.apply(calculate_price_per_m2_final, axis=1)
df['price_per_m2'].head()

0    375000.000000
1    490000.000000
2     72727.272727
3    500000.000000
4    485714.285714
Name: price_per_m2, dtype: float64

**Xử lý địa chỉ**

In [79]:
df.head(1)

,Title,Price,Area,Location,type,area_value,price_per_m2,price_value_trieu,price_per_m2_trieu,price_value_ty,price_per_m2_ty,price_value_nghin,price_per_m2_nghin
0,BÁN SẠP- KIOS CHỢ PHƯỜNG 4 TRÀ VINH,Giá: 9 triệu / tháng,Diện tích: 24 m\n2,"Đường Chu Văn An, Phường 4, Thành phố Trà Vinh...",cho_thue_cua_hang,24.0,375000.0,9.0,375000.0,None,NaN,NaN,NaN


In [80]:
df['address'] = df['Location']

In [90]:
df = df[['Title','Price','area_value','address','price_per_m2']]

In [82]:
list_api_keys = [
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"
]

In [83]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

In [84]:

def get_location(address, api_key):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [85]:
df_test = df

In [86]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [87]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Time: 4.883693778514862 phút
Time: 4.755802647272746 phút
Time: 4.8242961724599205 phút
Time: 4.752558366457621 phút
Time: 1.1045271317164103 phút
Done


In [104]:
merged_df = pd.concat(final, ignore_index=True)
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4225 entries, 0 to 4224
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         4225 non-null   object 
 1   Price         4225 non-null   object 
 2   area_value    4225 non-null   float64
 3   address       4225 non-null   object 
 4   area_value    4225 non-null   float64
 5   price_per_m2  4187 non-null   float64
 6   House_number  4225 non-null   object 
 7   Street        4225 non-null   object 
 8   Ward          4225 non-null   object 
 9   District      4225 non-null   object 
 10  City          4225 non-null   object 
 11  progress      4225 non-null   object 
dtypes: float64(3), object(9)
memory usage: 396.2+ KB


In [105]:
merged_df1 = merged_df[merged_df["progress"] != "False"]
merged_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4225 entries, 0 to 4224
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         4225 non-null   object 
 1   Price         4225 non-null   object 
 2   area_value    4225 non-null   float64
 3   address       4225 non-null   object 
 4   area_value    4225 non-null   float64
 5   price_per_m2  4187 non-null   float64
 6   House_number  4225 non-null   object 
 7   Street        4225 non-null   object 
 8   Ward          4225 non-null   object 
 9   District      4225 non-null   object 
 10  City          4225 non-null   object 
 11  progress      4225 non-null   object 
dtypes: float64(3), object(9)
memory usage: 429.1+ KB


In [106]:
merged_df1 = merged_df1.loc[:,~merged_df1.columns.duplicated()]

In [108]:
merged_df1 = merged_df1.rename(columns={'price_per_m2':'price_per_m2_final'})
merged_df1 = merged_df1[['City','District','Ward','Street','address','area_value','price_per_m2_final']]
merged_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4225 entries, 0 to 4224
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   City                4225 non-null   object 
 1   District            4225 non-null   object 
 2   Ward                4225 non-null   object 
 3   Street              4225 non-null   object 
 4   address             4225 non-null   object 
 5   area_value          4225 non-null   float64
 6   price_per_m2_final  4187 non-null   float64
dtypes: float64(2), object(5)
memory usage: 264.1+ KB


In [109]:
merged_df1 = merged_df1.dropna(subset=['price_per_m2_final'])

In [110]:
merged_df1['price_value_trieu'] = merged_df1['price_per_m2_final']*merged_df1['area_value']/1000000

In [111]:
final_alonhadat = merged_df1[['City','District','Ward','Street','address','area_value','price_value_trieu','price_per_m2_final']]
final_alonhadat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4187 entries, 0 to 4224
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   City                4187 non-null   object 
 1   District            4187 non-null   object 
 2   Ward                4187 non-null   object 
 3   Street              4187 non-null   object 
 4   address             4187 non-null   object 
 5   area_value          4187 non-null   float64
 6   price_value_trieu   4187 non-null   float64
 7   price_per_m2_final  4187 non-null   float64
dtypes: float64(3), object(5)
memory usage: 294.4+ KB


In [112]:
final_alonhadat.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bds_alonhadat.csv", index=False, mode = 'w', encoding="utf-8-sig")